In [1]:
class SearchEngineBase(object):
    def __init__(self):
        pass

    def add_corpus(self, file_path):
        with open(file_path, 'r') as fin:
            text = fin.read()
        self.process_corpus(file_path, text)

    def process_corpus(self, id, text):
        raise Exception('process_corpus not implemented.')

    def search(self, query):
        raise Exception('search not implemented.')

def main(search_engine):

    for file_path in ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt']:
        search_engine.add_corpus(fr'e:\git\learning\python\materials\ObjectOrientedProgramming_p2\{file_path}')

    while True:
        query = input()
        results = search_engine.search(query)
        print('found {} result(s):'.format(len(results)))
        for result in results:
            print(result)


In [3]:
class SimpleEngine(SearchEngineBase):
    def __init__(self):
        super(SimpleEngine, self).__init__()
        self.__id_to_texts = {}

    def process_corpus(self, id, text):
        self.__id_to_texts[id] = text

    def search(self, query):
        results = []
        for id, text in self.__id_to_texts.items():
            if query in text:
                results.append(id)
        return results

search_engine = SimpleEngine()
main(search_engine)



found 0 result(s):
found 2 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
found 0 result(s):
found 5 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\3.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\4.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\5.txt
found 5 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\3.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\4.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\5.txt
found 5 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:

`SimpleEngine` 实现了一个继承 `SearchEngineBase` 的子类，继承并实现了 `process_corpus` 和 `search` 接口，同时，也顺手继承了 `add_corpus` 函数。

在我们新的构造函数中，`self.__id_to_texts = {}` 初始化了自己的私有变量，也就是这个用来存储文件名到文件内容的字典。

`process_corpus()` 函数则非常直白地将文件内容插入到字典中。这里注意，ID 需要是唯一的，不然相同 ID 的新内容会覆盖掉旧的内容。

`search` 直接枚举字典，从中找到要搜索的字符串。如果能够找到，则将 ID 放到结果列表中，最后返回。

1. 现在你对父类子类的构造函数调用顺序和方法应该更清楚了吧？ 
2. 继承的时候，函数是如何重写的？ 
3. 基类是如何充当接口作用的（你可以自行删掉子类中的重写函数，抑或是修改一下函数的参数，看一下会报什么错）？ 
4. 方法和变量之间又如何衔接起来的呢？

相信你也能看得出来，这种实现方式简单，但显然是一种很低效的方式：每次索引后需要占用大量空间，因为索引函数并没有做任何事情；每次检索需要占用大量时间，因为所有索引库的文件都要被重新搜索一遍。如果把语料的信息量视为 n，那么这里的时间复杂度和空间复杂度都应该是 O(n) 级别的。 

而且，还有一个问题：这里的 query 只能是一个词，或者是连起来的几个词。如果你想要搜索多个词，它们又分散在文章的不同位置，我们的简单引擎就无能为力了。 

这时应该怎么优化呢？ 

最直接的一个想法，就是把语料分词，看成一个个的词汇，这样就只需要对每篇文章存储它所有词汇的 set 即可。根据齐夫定律（Zipf’s law，https://en.wikipedia.org/wiki/Zipf%27s_law），在自然语言的语料库里，一个单词出现的频率与它在频率表里的排名成反比，呈现幂律分布。因此，语料分词的做法可以大大提升我们的存储和搜索效率。 

那具体该如何实现呢？

## Bag of Words 和 Inverted Index

我们先来实现一个名叫 Bag of Words 的搜索模型。请看下面的代码：

In [3]:
import re

class BOWEngine(SearchEngineBase):
    def __init__(self):
        super(BOWEngine, self).__init__()
        self.__id_to_words = {}

    def process_corpus(self, id, text):
        self.__id_to_words[id] = self.parse_text_to_words(text)

    def search(self, query):
        query_words = self.parse_text_to_words(query)
        results = []
        for id, words in self.__id_to_words.items():
            if self.query_match(query_words, words):
                results.append(id)
        return results
    
    @staticmethod
    def query_match(query_words, words):
        for query_word in query_words:
            if query_word not in words:
                return False
        return True

    @staticmethod
    def parse_text_to_words(text):
        # 使用正则表达式去除标点符号和换行符
        text = re.sub(r'[^\w ]', ' ', text)
        # 转为小写
        text = text.lower()
        # 生成所有单词的列表
        word_list = text.split(' ')
        # 去除空白单词
        word_list = filter(None, word_list)
        # 返回单词的 set
        return set(word_list)

search_engine = BOWEngine()
main(search_engine)



found 5 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\3.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\4.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\5.txt
found 3 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\3.txt
found 1 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\5.txt
found 0 result(s):
found 5 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\3.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\4.txt
e:\git\learning\pytho

BOW Model，即 Bag of words Model, 中文叫做词袋模型。 

假设有一个文本，不考虑语法，句法，段落，也不考虑词汇出现的顺序，只将这个文本看成这些词汇的集合。所以在代码中将原有的`id_to_texts`替换成`id_to_words`，这样就只需要存这些单词，而不是全部文章，也不需要考虑顺序。其中， `process_corpus()` 函数调用类静态函数 `parse_text_to_words`，将文章打碎形成词袋，放入 `set` 之后再放到字典中。

`search()` 函数则稍微复杂一些。这里我们假设，想得到的结果，是所有的搜索关键词都要出现在同一篇文章中。那么，我们需要同样打碎 query 得到一个 set，然后把 set 中的每一个词，和我们的索引中每一篇文章进行核对，看一下要找的词是否在其中。而这个过程由静态函数 `query_match` 负责。

这样的解决方案存在一些问题：
1. 每次查询时依然需要遍历所有 ID，虽然比起 Simple 模型已经节约了大量时间，但是互联网上有上亿个页面，每次都全部遍历的代价还是太大了
2. 词袋模型并不考虑单词间的顺序，但有些人希望单词按顺序出现，或者希望搜索的单词在文中离得近一些，这种情况下词袋模型现任就无能为力了

针对以上两点可以做出以下改进：

In [5]:
import re

class BOWInvertedIndexEngine(SearchEngineBase):
    def __init__(self):
        super(BOWInvertedIndexEngine, self).__init__()
        self.inverted_index = {}

    def process_corpus(self, id, text):
        words = self.parse_text_to_words(text)
        for word in words:
            if word not in self.inverted_index:
                self.inverted_index[word] = []
            self.inverted_index[word].append(id)

    def search(self, query):
        query_words = list(self.parse_text_to_words(query))
        query_words_index = list()
        for query_word in query_words:
            query_words_index.append(0)
        
        # 如果某一个查询单词的倒序索引为空，我们就立刻返回
        for query_word in query_words:
            if query_word not in self.inverted_index:
                return []
        
        result = []
        while True:
            
            # 首先，获得当前状态下所有倒序索引的 index
            current_ids = []
            
            for idx, query_word in enumerate(query_words):
                current_index = query_words_index[idx]
                current_inverted_list = self.inverted_index[query_word]
                
                # 已经遍历到了某一个倒序索引的末尾，结束 search
                if current_index >= len(current_inverted_list):
                    return result

                current_ids.append(current_inverted_list[current_index])

            # 然后，如果 current_ids 的所有元素都一样，那么表明这个单词在这个元素对应的文档中都出现了
            if all(x == current_ids[0] for x in current_ids):
                result.append(current_ids[0])
                query_words_index = [x + 1 for x in query_words_index]
                continue
            
            # 如果不是，我们就把最小的元素加一
            min_val = min(current_ids)
            min_val_pos = current_ids.index(min_val)
            query_words_index[min_val_pos] += 1

    @staticmethod
    def parse_text_to_words(text):
        # 使用正则表达式去除标点符号和换行符
        text = re.sub(r'[^\w ]', ' ', text)
        # 转为小写
        text = text.lower()
        # 生成所有单词的列表
        word_list = text.split(' ')
        # 去除空白单词
        word_list = filter(None, word_list)
        # 返回单词的 set
        return set(word_list)

search_engine = BOWInvertedIndexEngine()
main(search_engine)



found 2 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\1.txt
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt
found 1 result(s):
e:\git\learning\python\materials\ObjectOrientedProgramming_p2\2.txt


IndexError: list index out of range

新模型继续使用之前的接口，仍然只在 `__init__()`、`process_corpus()`和`search()`三个函数进行修改。这其实也是大公司里团队协作的一种方式，在合理的分层设计后，每一层的逻辑只需要处理好分内的事情即可。在迭代升级我们的搜索引擎内核时， main 函数、用户接口没有任何改变。当然，如果公司招了新的前端工程师，要对用户接口部分进行修改，新人也不需要过分担心后台的事情，只要做好数据交互就可以了。

继续看代码，你可能注意到了开头的 Inverted Index。Inverted Index Model，即倒序索引，是非常有名的搜索引擎方法，接下来我简单介绍一下。

倒序索引，故如其名，也就是说这次反过来，我们保留的是 word -> id 的字典。于是情况就豁然开朗了，在 search 时，我们只需要把想要的 query_word 的几个倒序索引单独拎出来，然后从这几个列表中找共有的元素，那些共有的元素，即 ID，就是我们想要的查询结果。这样，我们就避免了将所有的 index 过一遍的尴尬。

process_corpus 建立倒序索引。注意，这里的代码都是非常精简的。在工业界领域，需要一个 unique ID 生成器，来对每一篇文章标记上不同的 ID，倒序索引也应该按照这个 unique_id 来进行排序。

至于 search() 函数，你大概了解它做的事情即可。它会根据 query_words 拿到所有的倒序索引，如果拿不到，就表示有的 query word 不存在于任何文章中，直接返回空；拿到之后，运行一个“合并 K 个有序数组”的算法，从中拿到我们想要的 ID，并返回。

注意，这里用到的算法并不是最优的，最优的写法需要用最小堆来存储 index。这是一道有名的 leetcode hard 题，有兴趣请参考：https://blog.csdn.net/qqxx6661/article/details/77814794）

遍历的问题解决了，那第二个问题，如果我们想要实现搜索单词按顺序出现，或者希望搜索的单词在文中离得近一些呢？ 我们需要在 Inverted Index 上，对于每篇文章也保留单词的位置信息，这样一来，在合并操作的时候处理一下就可以了。

## LRU和多重继承



到这一步，终于，你的搜索引擎上线了，有了越来越多的访问量（QPS）。欣喜骄傲的同时，你却发现服务器有些“不堪重负”了。经过一段时间的调研，你发现大量重复性搜索占据了 90% 以上的流量，于是，你想到了一个大杀器——给搜索引擎加一个缓存。

In [ ]:
import pylru

class LRUCache(object):
    def __init__(self, size=32):
        self.cache = pylru.lrucache(size)
    
    def has(self, key):
        return key in self.cache
    
    def get(self, key):
        return self.cache[key]
    
    def set(self, key, value):
        self.cache[key] = value

class BOWInvertedIndexEngineWithCache(BOWInvertedIndexEngine, LRUCache):
    def __init__(self):
        super(BOWInvertedIndexEngineWithCache, self).__init__()
        LRUCache.__init__(self)
    
    def search(self, query):
        if self.has(query):
            print('cache hit!')
            return self.get(query)
        
        result = super(BOWInvertedIndexEngineWithCache, self).search(query)
        self.set(query, result)
        
        return result

search_engine = BOWInvertedIndexEngineWithCache()
main(search_engine)



它的代码很简单，LRUCache 定义了一个缓存类，你可以通过继承这个类来调用其方法。LRU 缓存是一种很经典的缓存（同时，LRU 的实现也是硅谷大厂常考的算法面试题，这里为了简单，我直接使用 pylru 这个包），它符合自然界的局部性原理，可以保留最近使用过的对象，而逐渐淘汰掉很久没有被用过的对象。

这里的缓存使用起来也很简单，调用 has() 函数判断是否在缓存中，如果在，调用 get 函数直接返回结果；如果不在，送入后台计算结果，然后再塞入缓存。

BOWInvertedIndexEngineWithCache 类，多重继承了两个类。多重继承有两种：

第一种方法，用下面这行代码，直接初始化该类的第一个父类：`super(BOWInvertedIndexEngineWithCache, self).__init__()`,不过使用这种方法时，要求继承链的最顶层父类必须要继承 object。


第二种方法，对于多重继承，如果有多个构造函数需要调用， 我们必须用传统的方法L`RUCache.__init__(self)` 。你应该注意，search() 函数被子类 BOWInvertedIndexEngineWithCache 再次重载，但是我还需要调用 BOWInvertedIndexEngine 的 search() 函数，这时该怎么办呢？请看下面这行代码：`super(BOWInvertedIndexEngineWithCache, self).search(query)`。我们可以强行调用被覆盖的父类的函数。
